In [36]:
# all imports
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd

In [37]:
# define directories
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/' # my cache path
vi_dir = '/pscratch/sd/j/jjpim/visual_inspection/' # vi image path

#constants
c = 299792.458

In [38]:
# load updated fuji table
tf_fuji = Table.read(data_dir + 'SV/desi_pv_tf_fuji_healpix_rotcurve_EOA_VI.fits') 
# load iron table
tf_iron = Table.read(data_dir + 'Y1/desi_pv_tf_iron_healpix_rotcurve_EOA_VI.fits')
#combine
tf_mastertable = vstack([tf_fuji,tf_iron])

In [39]:
#mastertable columns
tf_mastertable['VELOCITY']=np.nan
tf_mastertable['VERR']=np.nan
tf_mastertable['WEIGHTED_CENTER']=np.nan

In [40]:
#create new columns for fuji
tf_fuji['VELOCITY']=np.nan
tf_fuji['VERR']=np.nan
tf_fuji['WEIGHTED_CENTER']=np.nan

#create new columns for iron
tf_iron['VELOCITY']=np.nan
tf_iron['VERR']=np.nan
tf_iron['WEIGHTED_CENTER']=np.nan

In [41]:
# set up dictionary of unique SGA IDs
tf_fuji_dict = {}
for sga_id in np.unique(tf_fuji['SGA_ID']):
    tf_fuji_dict[sga_id] = np.where(tf_fuji['SGA_ID'] == sga_id)[0]

tf_iron_dict = {}
for sga_id in np.unique(tf_iron['SGA_ID']):
    tf_iron_dict[sga_id] = np.where(tf_iron['SGA_ID'] == sga_id)[0]

tf_mastertable_dict = {}
for sga_id in np.unique(tf_mastertable['SGA_ID']):
    tf_mastertable_dict[sga_id] = np.where(tf_mastertable['SGA_ID'] == sga_id)[0]

Below is the code achieved w/ prof. douglass, uses mastertable instead of seperate data sets. Updating fuji and iron separately after.

In [42]:
# combine iron and fuji unique IDs
tf_galaxies = np.unique(tf_mastertable['SGA_ID'])

# loop through each SGA ID
for sga_id in tf_galaxies:
                            
    #identify z values of targets
    z_targets=tf_mastertable['Z'][tf_mastertable_dict[sga_id]]
    
    #identify good centers
    centers_b = tf_mastertable['SKY_FIBER_DIST_R26'][tf_mastertable_dict[sga_id]] < 0.001
    good_b = (tf_mastertable['ZWARN'][tf_mastertable_dict[sga_id]]==0) & (tf_mastertable['DELTACHI2'][tf_mastertable_dict[sga_id]]>25)
    good_centers = tf_mastertable[tf_mastertable_dict[sga_id]][centers_b & good_b] 
    
    #defining terms
    z = good_centers['Z']
    z_err_c = good_centers['ZERR']
    # calculating the weights
    #w = np.sum(w_fuji + w_iron)
    w = np.sum(1 / (z_err_c**2))
    
    # calculating the weighted average for center z naught
    z_o = (np.nanmean(w * z)) / w
    
    #define z naught error
    z_o_err = np.sqrt(1 / w)
    
    #find delta z from z naught and z values of all viable targets 
    delta_z = z_targets - z_o
    
    #use delta z to find velocity
    velocity = c * delta_z
    # update the master table with velocities
    tf_mastertable['VELOCITY'][tf_mastertable_dict[sga_id]]=velocity
    
    #identify z error for targets (used in v error calculation)
    z_err = tf_mastertable['ZERR'][tf_mastertable_dict[sga_id]]
    
    #find v error for each target
    v_error_1 = ((z_o_err**2) + (z_err**2))
    v_err=c*(np.sqrt(v_error_1))
    
    # put v error into new column
    tf_mastertable['VERR'][tf_mastertable_dict[sga_id]]=v_err

/tmp/ipykernel_171020/4109031097.py:23: RuntimeWarning: Mean of empty slice
  z_o = (np.nanmean(w * z)) / w
/tmp/ipykernel_171020/4109031097.py:26: RuntimeWarning: divide by zero encountered in double_scalars
  z_o_err = np.sqrt(1 / w)


In [50]:
#use mastertable velocities to update fuji table
'''for targetid in range(len(tf_fuji)):
    tf_fuji.loc[targetid, 'VELOCITY'] = [targetid]
    '''  
'''for i in range(len(tf_fuji)):
    i_mastertable=tf_fuji['TARGETID'][i]==tf_mastertable['TARGETID'][:len(tf_fuji)]
    tf_fuji['VELOCITY'][i]=tf_mastertable['VELOCITY'][:len(tf_fuji)][i_mastertable]
    tf_fuji['VERR'][i]=tf_mastertable['VERR'][:len(tf_fuji)][i_mastertable]
'''
tf_fuji['VELOCITY']=tf_mastertable['VELOCITY'][:len(tf_fuji)]
tf_fuji['VERR']=tf_mastertable['VERR'][:len(tf_fuji)]

In [49]:
#updating iron table with velocity and velocity error
tf_iron['VELOCITY']=tf_mastertable['VELOCITY'][-len(tf_iron):]
tf_iron['VERR']=tf_mastertable['VERR'][-len(tf_iron):]

In [45]:
tf_fuji[tf_fuji_dict[123]]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,WEIGHTED_CENTER
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1071233026228227,242.564878755415,53.6763270782975,10153,sv3,0.184971788085982,4.36696398757541e-05,4,0.458266502246261,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56487875541504,53.67632707829753,1.0,0.0051345584203947876,1.0000000466304622,1.0,0.0,22486.14009337598,13.370585897277147,nan
1071233026228228,242.564908359756,53.6746327647682,10153,sv3,1.5161235823035,9.98100168252869e-05,4,0.224537968635559,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.5649083597559,53.67463276476823,1.0,0.0034401541416619126,0.6700000312419042,1.0,0.0,421555.40845295583,30.045299874011604,nan
1071233026228231,242.565028546859,53.6677528249076,10153,sv3,0.223582059648363,6.6695997421467e-05,4,5.54895717371255,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56502854685928,53.667752824907645,1.0,0.0034401541416594996,0.6700000312414343,1.0,0.0,34061.20830910968,20.178575141850796,nan
1084427165761544,242.565058139153,53.6660585113414,10153,sv3,0.125989095083328,3.5227684897735e-05,4,5.68882738612592,fuji/healpix/sv3/dark/101/10153/redrock-sv3-dark-10153.fits,TFT,123,242.56505813915317,53.66605851134143,1.0,0.0051345584203787275,1.0000000466273342,1.0,0.0,4803.573578650942,10.904640688244093,nan
39633304835591852,242.564968458214,53.671192794853,10153,sv3,0.109965464429358,1.30236298072821e-05,0,847.705579817295,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56496845821405,53.67119279485298,1.0,1.6837541040866038e-14,3.2792580096330368e-12,1.0,0.0,-0.19004118687180754,4.756134887529947,nan


In [46]:
tf_iron[tf_iron_dict[123]]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,WEIGHTED_CENTER
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
39633304835591852,242.564968458214,53.671192794853,10153,sv3,0.109966732247689,1.26106529808024e-05,0,879.694150358438,iron/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56496845821405,53.67119279485298,1.0,1.6837541040866038e-14,3.2792580096330368e-12,1.0,0.0,0.190041186875968,4.655036418078015,nan


In [47]:
#testing output for sga id 123
tf_mastertable[tf_mastertable_dict[123]]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,WEIGHTED_CENTER
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1071233026228227,242.564878755415,53.6763270782975,10153,sv3,0.184971788085982,4.36696398757541e-05,4,0.458266502246261,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56487875541504,53.67632707829753,1.0,0.0051345584203947876,1.0000000466304622,1.0,0.0,22486.14009337598,13.370585897277147,nan
1071233026228228,242.564908359756,53.6746327647682,10153,sv3,1.5161235823035,9.98100168252869e-05,4,0.224537968635559,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.5649083597559,53.67463276476823,1.0,0.0034401541416619126,0.6700000312419042,1.0,0.0,421555.40845295583,30.045299874011604,nan
1071233026228231,242.565028546859,53.6677528249076,10153,sv3,0.223582059648363,6.6695997421467e-05,4,5.54895717371255,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56502854685928,53.667752824907645,1.0,0.0034401541416594996,0.6700000312414343,1.0,0.0,34061.20830910968,20.178575141850796,nan
1084427165761544,242.565058139153,53.6660585113414,10153,sv3,0.125989095083328,3.5227684897735e-05,4,5.68882738612592,fuji/healpix/sv3/dark/101/10153/redrock-sv3-dark-10153.fits,TFT,123,242.56505813915317,53.66605851134143,1.0,0.0051345584203787275,1.0000000466273342,1.0,0.0,4803.573578650942,10.904640688244093,nan
39633304835591852,242.564968458214,53.671192794853,10153,sv3,0.109965464429358,1.30236298072821e-05,0,847.705579817295,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56496845821405,53.67119279485298,1.0,1.6837541040866038e-14,3.2792580096330368e-12,1.0,0.0,-0.19004118687180754,4.756134887529947,nan
39633304835591852,242.564968458214,53.671192794853,10153,sv3,0.109966732247689,1.26106529808024e-05,0,879.694150358438,iron/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56496845821405,53.67119279485298,1.0,1.6837541040866038e-14,3.2792580096330368e-12,1.0,0.0,0.190041186875968,4.655036418078015,nan


In [48]:
tf_fuji.write(data_dir + 'SV/desi_pv_tf_fuji_healpix_rotcurve_EOA_VI_velocity.fits', format='fits', overwrite=True)
tf_iron.write(data_dir + 'Y1/desi_pv_tf_iron_healpix_rotcurve_EOA_VI_velocity.fits', format='fits', overwrite=True)